<a href="https://colab.research.google.com/github/badcortex/opt4ds/blob/master/rna_folding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In order to run the following notebook it is necessary to install the *Pyomo* library.  

In [1]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

     |████████████████████████████████| 2.5MB 9.6MB/s 
     |████████████████████████████████| 266kB 48.7MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 163kB 52.5MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 144429 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-

# The RNA Folding problem 

**The Simple RNA Folding problem** : Given the nucleotide sequence *s* of a RNA molecule, find a *nested pairing* that pairs the *maximum* number of nucleotides, compared to any other nested pairing. 

### A toy example
The ILP formulation for the Simple RNA Folding problem will have one binary variable, called $P(i,j)$, for each pair $(i,j)$ of positions in *s*, where $i < j$. The value $P(i,j)$ given by a feasible solution to the ILP formulation whether or not the nucleotide in $i$ of *s* will be paired with the nucleotide in position $j$ of *s*. 

1. **The objective funtion** : $$ Maximize \sum_{i<j} P(i,j) $$ 
2. **The inequalities** : For every pair $(i,j)$ of positions in *s* that do not have complementary characters $$ P(i,j) = 0 $$ 
For each position $j$ in *s* $$ \sum_{k < j} P(k,j) + \sum_{k > j} P(j,k) \leq 1 $$ For every choice of four positions $i < i'< j < j'$
$$ P(i,j)+P(i',j') \leq 1 $$



In [0]:
s = "ACUGU" # RNA molecule string

from pyomo.environ import *

# Creazione del modello 
model = ConcreteModel()

# Indici 
model.i = RangeSet(1,len(s))
model.j = RangeSet(1,len(s))

# Variabile
model.p = Var(model.i, model.j, within=Binary)

# Funzione obiettivo
model.obj = Objective(expr=model.p[1,3]+model.p[1,5]+model.p[2,4], sense=maximize)

# Vincoli 
model.pair1 = Constraint(expr = model.p[1,2]+model.p[1,3]+model.p[1,4]+model.p[1,5] <= 1)
model.pair2 = Constraint(expr = model.p[1,2]+model.p[2,3]+model.p[2,4]+model.p[2,5] <= 1)
model.pair3 = Constraint(expr = model.p[1,3]+model.p[2,3]+model.p[3,4]+model.p[3,5] <= 1)
model.pair4 = Constraint(expr = model.p[1,4]+model.p[2,4]+model.p[3,4]+model.p[4,5] <= 1)
model.pair5 = Constraint(expr = model.p[1,5]+model.p[2,5]+model.p[3,5]+model.p[4,5] <= 1)

model.nest1 = Constraint(expr = model.p[1,3]+model.p[2,4] <= 1)
model.nest2 = Constraint(expr = model.p[1,3]+model.p[2,5] <= 1)
model.nest3 = Constraint(expr = model.p[1,4]+model.p[2,5] <= 1)
model.nest4 = Constraint(expr = model.p[1,4]+model.p[3,5] <= 1)
model.nest5 = Constraint(expr = model.p[2,4]+model.p[3,5] <= 1)

In [3]:
# Solve the model
sol = SolverFactory('glpk').solve(model)

# Basic info about the solution process
for info in sol['Solver']:
    print(info)


Status: ok
Termination condition: optimal
Statistics: 
  Branch and bound: 
    Number of bounded subproblems: 1
    Number of created subproblems: 1
Error rc: 0
Time: 0.010518074035644531



In [4]:
# Report solution value
print("Optimal solution value: z =", model.obj())

Optimal solution value: z = 2.0


In [8]:
# Report solution value
print("Optimal solution value: z =", model.obj())
print("Decision variables:")
for i in range(1,len(s)):
    for j in range(i,len(s)+1):
        print("P({},{}) = {}".format(i, j, model.p[i,j]()))

Optimal solution value: z = 2.0
Decision variables:
P(1,1) = None
P(1,2) = 0.0
P(1,3) = 0.0
P(1,4) = 0.0
P(1,5) = 1.0
P(2,2) = None
P(2,3) = 0.0
P(2,4) = 1.0
P(2,5) = 0.0
P(3,3) = None
P(3,4) = 0.0
P(3,5) = 0.0
P(4,4) = None
P(4,5) = 0.0
